# Rapport TAL
## Partie Présidents
Membres du binôme :
- KRISNI Almehdi (3800519)
- ARICHANDRA Santhos (3802651)

### Introduction
Dans cette partie, nous allons nous intéresser à la prédiction de l'interlocuteur lors d'un débat présidentiel entre Chirac et Mitterrand. Entre d'autres termes, qui a dit quoi ?

Nous disposons donc d'une base de données d'apprentissage de 57000 lignes représentant un ensemble de phrases, chacune de ces dernières étiquetées par un C (Chirac) ou un M (Mitterrand), désignant la personne l'ayant prononcé. On peut alors labéliser notre ensemble d'apprentissage en 2 classes distinctes. 

Notre objectif est de mettre en place des algorithmes allant nous permettre de labelliser de nouveaux débats entre Chirac et Mitterrand.

### Mise en place des import et des librairies utilisées

In [168]:
import numpy as np
import matplotlib.pyplot as plt

import codecs
import re
import os.path

import copy

from collections import Counter

import re
import unicodedata
import string
from nltk.corpus import stopwords
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop

cachedStopWords = stopwords.words("french") + (list(fr_stop))
print(cachedStopWords)

from collections import Counter

from sklearn.feature_extraction import text 

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import numpy as np
import sklearn.naive_bayes as nb
from sklearn import svm
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV

# Importation des librairies standards:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  

# Importation de votre librairie iads
import sys
sys.path.append('../')

# Iportation de utils
from iads import utils as ut

# commande TRES utile pour recharger automatiquement le code que vous modifiez dans les modules
%load_ext autoreload
%autoreload 2

['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils', 'je', 'la', 'le', 'les', 'leur', 'lui', 'ma', 'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne', 'nos', 'notre', 'nous', 'on', 'ou', 'par', 'pas', 'pour', 'qu', 'que', 'qui', 'sa', 'se', 'ses', 'son', 'sur', 'ta', 'te', 'tes', 'toi', 'ton', 'tu', 'un', 'une', 'vos', 'votre', 'vous', 'c', 'd', 'j', 'l', 'à', 'm', 'n', 's', 't', 'y', 'été', 'étée', 'étées', 'étés', 'étant', 'étante', 'étants', 'étantes', 'suis', 'es', 'est', 'sommes', 'êtes', 'sont', 'serai', 'seras', 'sera', 'serons', 'serez', 'seront', 'serais', 'serait', 'serions', 'seriez', 'seraient', 'étais', 'était', 'étions', 'étiez', 'étaient', 'fus', 'fut', 'fûmes', 'fûtes', 'furent', 'sois', 'soit', 'soyons', 'soyez', 'soient', 'fusse', 'fusses', 'fût', 'fussions', 'fussiez', 'fussent', 'ayant', 'ayante', 'ayantes', 'ayants', 'eu', 'eue', 'eues', 'eus', 'ai', 'as', 'avons', 'avez', 'ont', 'aurai', 'auras', 'aura', 'aurons', 'aur

### Première analyse des données

On extrait dans un premier les informations à la base de données d'apprentissage. On utilise la fonction d'extraction "load_pres" située dans le fichier utils.py.

On considère 2 classes, la classe -1 représentant Mitterrand et la classe +1 représentant Chirac.

In [169]:
appdata = ut.load_pres("AFDpresidentutf8/corpus.tache1.learn.utf8")
appX = appdata[0]
appY = appdata[1]

On s'intéresse à la répartition des labels dans les données. Une personne parle-t-elle beaucoup plus que l'autre ?

In [170]:
print("Le nombre de lignes prononcées par Chirac :\t", str(appY.count(1)))
print("Le nombre de lignes prononcées par Mitterrand :\t", str(appY.count(-1)), "\n")
print("Proportion de Chirac dans la base d'apprentissage :\t", str((appY.count(1) / len(appY)) * 100), "%")
print("Proportion de Mitterrand dans la base d'apprentissage :\t", str((appY.count(-1) / len(appY)) * 100), "%")

Le nombre de lignes prononcées par Chirac :	 49890
Le nombre de lignes prononcées par Mitterrand :	 7523 

Proportion de Chirac dans la base d'apprentissage :	 86.89669587027329 %
Proportion de Mitterrand dans la base d'apprentissage :	 13.103304129726718 %


On remarque donc qu'il y a presque 87% des lignes dans la base de données d'apprentissage ayant été prononcées par Mitterrand. Il se peut que cette différence puisse jouer un rôle important lors de l'apprentissage.

### Nettoyage des données

On effectue alors un premier nettoyage des données en supprimant les chiffres, la ponctuation, les accents, les majuscules et les stopwords de la langue française.

In [177]:
appX = appdata[0]
appY = appdata[1]
appX = ut.transform(appX, punc=True, accentMaj=True, nb=True, stopW=True, base=cachedStopWords)

On crée un Counter allant mettre en avant les mots les plus souvent utilisés par chacun des Présidents, après nettoyage des données en fonction des paramètres selectionnés.

In [178]:
# Création du dictionnaire et counter de mots de Mitterrand
motsM = [appX[i] for i in range(len(appY)) if appY[i] == -1]
motsM = (' '.join(motsM)).split()
counterM = Counter(motsM)

print("Les 10 mots les plus utilisés par Mitterrand sont :\n", [i[0] for i in counterM.most_common(10)], "\n")

# Creation du dictionnaire et counter de mots de Chirac
motsC = [appX[i] for i in range(len(appY)) if appY[i] == 1]
motsC = (' '.join(motsC)).split()
counterC = Counter(motsC)

print("Les 10 mots les plus utilisés par Chirac sont :\n", [i[0] for i in counterC.most_common(10)])

Les 10 mots les plus utilisés par Mitterrand sont :
 ['bien', 'pays', 'france', 'faut', 'europe', 'ete', 'nom', 'date', 'faire', 'temps'] 

Les 10 mots les plus utilisés par Chirac sont :
 ['france', 'pays', 'nom', 'aujourd', 'monde', 'francais', 'bien', 'europe', 'faire', 'etat']


On s'intéresse à mettre en place deux dictionnaires exclusifs pour Chirac et Mitterrand. Il s'agit de dictionnaires comportant les mots qu'ils utilisent de manière exclusive.

In [186]:
# Dictionnaire exclusif de Mitterrand
excluM = copy.deepcopy(counterM)
for w in counterC.keys() :
    if w in excluM.keys() :
        del excluM[w]
        
print("Le ratio de mots exclusifs utilisés par Mitterrand est de :\t", len(excluM.keys()) / len(counterM.keys()))
print("Les 10 mots exclusifs les plus utilisés par Mitterrand sont :\n", [i[0] for i in excluM.most_common(10)], "\n")

# Dictionnaire exclusif de Chirac
excluC = copy.deepcopy(counterC)
for w in counterM.keys() :
    if w in excluC.keys() :
        del excluC[w]
        
print("Le ratio de mots exclusifs utilisés par Chirac est de :\t\t", len(excluC.keys()) / len(counterC.keys()))
print("Les 10 mots exclusifs les plus utilisés par Chirac sont :\n", [i[0] for i in excluC.most_common(10)])

Le ratio de mots exclusifs utilisés par Mitterrand est de :	 0.2238770870894411
Les 10 mots exclusifs les plus utilisés par Mitterrand sont :
 ['convenait', 'eureka', 'nievre', 'morvan', 'estampes', 'cee', 'nevers', 'laitiers', 'electrification', 'montlucon'] 

[('ranimee', 1), ('reliefs', 1), ('allegories', 1), ('rechigner', 1), ('regroupaient', 1), ('geostrategie', 1), ('mappemonde', 1), ('agglomerer', 1), ('adviendra', 1), ('malhonnete', 1), ('receptivite', 1), ('montebello', 1), ('etriques', 1), ('avantageuse', 1), ('depeupler', 1), ('depeuple', 1), ('tsar', 1), ('miradors', 1), ('comique', 1), ('reactivee', 1), ('deplairait', 1), ('desagrement', 1), ('apercevra', 1), ('embellit', 1), ('retenons', 1), ('inegaux', 1), ('paraisse', 1), ('privielegiees', 1), ('souffrirons', 1), ('vivifiees', 1), ('respecterai', 1), ('cederai', 1), ('isolant', 1), ('rejointes', 1), ('dan', 1), ('ajouteriez', 1), ('restreignant', 1), ('repeterons', 1), ('connaitrons', 1), ('anterieurement', 1), ('depass